![glassdoor](https://github.com/jasonchanhku/DataScienceDemand/blob/master/images/glassdoor.png?raw=true)

# Data Science Demand in Hong Kong
### by Jason Chan Jin An

# Introduction

This project aims to gauge the data science job demands in Hong Kong
in the past 30 days rolling based on job posts from Glassdoor. Glassdoor was 
the preferred data source because its wide array of available information:

* Job Title
* Company Name
* Link
* Company Rating
* Job Description
* Company Size
* Year Founded
* Company Type
* Industry
* Company Revenue
* CEO (sentiment)
* Recommend Percentage
* Approval Percentage

# Questions to be Answered

Exploratory Data Analysis (EDA) is performed in this notebook from a statistical and data standpoint and seeks to answer the following questions: 
* Who are hiring data scientists in Hong Kong ?
    * Big / small companies ?
    * What kind of industries ?
    * Good company feedback and approval ?
* Do company ratings differ from company types / industry / etc ?
* Given my preference of company type, which jobs suits me most ?

## Dataset

The dataset was obtained by building a python web scraper using `selenium` and the script is saved as `scraper.py` in the repository.

***

# Data Prep and Libraries

In [1]:
# Libraries used
import pandas as pd
import numpy as np

In [3]:
# Data prep
df = pd.read_csv('https://raw.githubusercontent.com/jasonchanhku/DataScienceDemand/master/data/glassdoor_data.csv')
df.head(5)

Title           Company  \
0  CIB QR - Risk Quantitative Research, Equity De...       J.P. Morgan   
1                    Quantitative Research - M/F VIE  Societe Generale   
2                            Data Analyst - Modeling        Transunion   
3                                     Data Scientist            Lenovo   
4                            Quantitative Researcher  Societe Generale   

                                                Link  Rating  \
0  https://www.glassdoor.com/partner/jobListing.h...     3.7   
1  https://www.glassdoor.com/partner/jobListing.h...     3.4   
2  https://www.glassdoor.com/partner/jobListing.h...     3.9   
3  https://www.glassdoor.com/partner/jobListing.h...     3.3   
4  https://www.glassdoor.com/partner/jobListing.h...     3.4   

                                     Job_Description                    Size  \
0  J.P. Morgans Corporate & Investment Bank is a ...        10000+ employees   
1  Environment\n\nYour environment\nSG CIB is the...        10000+ employees   
2  Dynamics of the Role\n\nThe incumbent\nis expe...  1001 to 5000 employees   
3  Position Description\nDesign data mining and m...        10000+ employees   
4  Environment\n\nSG CIB is the Corporate and Inv...        10000+ employees   

  Founded    Company_Type                Industry  \
0    1799    Public (JPM)                 Finance   
1    1864    Public (GLE)                 Finance   
2    1968    Public (TRU)                 Finance   
3    1984  Public (LNVGY)  Information Technology   
4    1864    Public (GLE)                 Finance   

                           Revenue             CEO  Recommend  Approve  
0      $10+ billion (USD) per year     Jamie Dimon       76.0     93.0  
1      $10+ billion (USD) per year  Frederic Oudea       68.0     83.0  
2  $1 to $2 billion (USD) per year        Jim Peck       75.0     93.0  
3      $10+ billion (USD) per year   Yang Yuanqing       57.0     64.0  
4      $10+ billion (USD) per year  Frederic Oudea       68.0     83.0

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 13 columns):
Title              211 non-null object
Company            211 non-null object
Link               211 non-null object
Rating             189 non-null float64
Job_Description    211 non-null object
Size               207 non-null object
Founded            207 non-null object
Company_Type       207 non-null object
Industry           207 non-null object
Revenue            207 non-null object
CEO                190 non-null object
Recommend          190 non-null float64
Approve            190 non-null float64
dtypes: float64(3), object(10)
memory usage: 21.5+ KB


# Data Preprocessing

## Exclude the following rows:
* Rows of comapny that is 'RegionUP' as they have been flagged as a scam company
* Drop rows with NA columns since insignificnat, will make it easier for machine learning
* Drop rows without CEO approval ratings since very small amount of them do not have

## Basic cleaning of Job_Description column:
* Strip leading and trailing whitespaces
* Strip '\n' from the job column description and other html tags
* Replace double space with single space

## Improve readability of revenue column
* Remove unecessary parts of text
* Impute values of 'Unknown / Not Applicable' using employee size
* Preserve ordinality by mapping

## Impute Size column
* Impute using revenue column

In [8]:
# Exclude outlier RegionUP as it is a scam company
df = df[df['Company'] != 'RegionUP']

# Drop NA values
df = df.dropna()

# Drop those without Approval scores
df = df[df['Approve'] != -1]

In [33]:
# Cleanse job description column
def jd_cleanse(col):
    
    col = col.apply(lambda x: x.strip())
    col = col.apply(lambda x: x.replace('\n', ' '))
    col = col.apply(lambda x: x.replace('  ', ' '))
    
    return col

# Cleanse the revenue column

def rev_cleanse(col):
    
    col = col.apply(lambda x: x.replace('per year', ''))
    col = col.apply(lambda x: x.replace('(USD)', ''))
    col = col.apply(lambda x: x.strip())
    
    return col
    

### Job_Description Column

In [27]:
df['Job_Description'] = jd_cleanse(df['Job_Description'])

In [36]:
df['Job_Description'].head()

0    J.P. Morgans Corporate & Investment Bank is a ...
1    Environment Your environment SG CIB is the Cor...
2    Dynamics of the Role The incumbent is expected...
3    Position Description Design data mining and ma...
4    Environment SG CIB is the Corporate and Invest...
Name: Job_Description, dtype: object

### Revenue Column

In [37]:
df['Revenue'] = rev_cleanse(df['Revenue'])

How many are Unknown / Not Applicable ?

In [40]:
df['Revenue'].value_counts()

$10+ billion                  57
Unknown / Non-Applicable      21
$5 to $10 billion             18
$1 to $2 billion               8
$2 to $5 billion               8
$50 to $100 million            7
$100 to $500 million           7
$10 to $25 million             4
$500 million to $1 billion     4
$1 to $5 million               2
Less than $1 million           2
$25 to $50 million             2
$5 to $10 million              1
Name: Revenue, dtype: int64

who are the unknown Revenue companies ?

In [62]:
df[df['Revenue'] == 'Unknown / Non-Applicable'][['Company', 'Size']]

Company                     Size
12                Bloomberg         10000+ employees
14       Compare Asia Group      51 to 200 employees
15                Bloomberg         10000+ employees
20                Bloomberg         10000+ employees
24                DataRobot      51 to 200 employees
25         CompareAsiaGroup      51 to 200 employees
29                     Lynk      51 to 200 employees
34                Bloomberg         10000+ employees
42                     Lynk      51 to 200 employees
48       Compare Asia Group      51 to 200 employees
52                Bloomberg         10000+ employees
55                Bloomberg         10000+ employees
56                     Lynk      51 to 200 employees
60                   Axalta         10000+ employees
63   Fidelity International  5001 to 10000 employees
65                Tink labs     201 to 500 employees
138         Hamlyn Williams      51 to 200 employees
148    DST Global Solutions   1001 to 5000 employees
164      Hong Kong Airlines    501 to 1000 employees
166      Compare Asia Group      51 to 200 employees
170        CompareAsiaGroup      51 to 200 employees

## Imputing Revenue and Size by most common occrence

* It is possible to impute missing revenu and size based on the following pivot table
    * As most unknown size has a revenue of   


### Pivot Table of Revenue mode occurence for each Size

In [63]:
mapping = pd.pivot_table(df[['Size', 'Revenue']],index=['Size'], columns=['Revenue'], aggfunc=np.count_nonzero).idxmax(axis =1)

In [65]:
dict(mapping)

{'1 to 50 employees': '$10 to $25 million',
 '10000+ employees': '$10+ billion',
 '1001 to 5000 employees': '$1 to $2 billion',
 '201 to 500 employees': '$10+ billion',
 '5001 to 10000 employees': '$5 to $10 billion',
 '501 to 1000 employees': '$100 to $500 million',
 '51 to 200 employees': 'Unknown / Non-Applicable',
 'Unknown': '$5 to $10 million'}